In [ ]:
import pandas as pd
import numpy as np
from bokeh.plotting import figure, show, output_notebook
from bokeh.models import ColumnDataSource
output_notebook()

df = pd.read_csv('/content/dataset(1).csv')


df.rename(columns={
    'Capacity': 'capacity',
    'Occupancy': 'occupancy',
    'QueueLength': 'queue_length',
    'TrafficConditionNearby': 'traffic',
    'IsSpecialDay': 'is_special_day',
    'VehicleType': 'vehicle_type',
    'LastUpdatedDate': 'date',
    'LastUpdatedTime': 'time'
}, inplace=True)


df['traffic'] = pd.to_numeric(df['traffic'], errors='coerce')
df['queue_length'] = pd.to_numeric(df['queue_length'], errors='coerce')
df['is_special_day'] = pd.to_numeric(df['is_special_day'], errors='coerce')
df['occupancy'] = pd.to_numeric(df['occupancy'], errors='coerce')
df['capacity'] = pd.to_numeric(df['capacity'], errors='coerce')


df['timestamp'] = pd.to_datetime(df['date'] + ' ' + df['time'], dayfirst=True, errors='coerce')

df.sort_values('timestamp', inplace=True)

vehicle_weights = {'car': 1.0, 'bike': 0.8, 'truck': 1.2}

def baseline_model(price_t, occupancy, capacity, alpha=0.1):
    utilization = occupancy / capacity if capacity > 0 else 0
    return price_t + alpha * utilization

def calculate_demand(row):
    vtype = str(row['vehicle_type']).lower()
    demand = (
        0.4 * (row['occupancy'] / row['capacity']) +
        0.2 * row['queue_length'] -
        0.15 * row['traffic'] +
        0.1 * row['is_special_day'] +
        0.15 * vehicle_weights.get(vtype, 1.0)
    )
    return demand

def demand_model(base_price, demand, λ=0.3):
    demand = np.clip(demand, 0, 2)
    price = base_price * (1 + λ * (demand - 1))
    return np.clip(price, base_price * 0.5, base_price * 2.0)

base_price = 10
price_t = base_price
baseline_prices = []

for _, row in df.iterrows():
    price_t = baseline_model(price_t, row['occupancy'], row['capacity'])
    baseline_prices.append(price_t)

df['price_baseline'] = baseline_prices
df['demand'] = df.apply(calculate_demand, axis=1)
df['price_demand'] = [demand_model(base_price, d) for d in df['demand']]

source = ColumnDataSource(df)

p = figure(x_axis_type="datetime", title="Dynamic Parking Lot Pricing", width=750, height=400)
p.line(x='timestamp', y='price_baseline', source=source, color='green', legend_label='Baseline Model', line_width=2)
p.line(x='timestamp', y='price_demand', source=source, color='blue', legend_label='Demand-Based Model', line_width=2)

p.xaxis.axis_label = "Time"
p.yaxis.axis_label = "Price ($)"
p.legend.location = "top_left"

show(p)


Summer Analytics 2025 – Capstone Project
Organized by C&A Club × Pathway
What I Did

In this project, I made a system that changes parking prices during the day based on how full the lot is, how much traffic is nearby, what kind of vehicle is coming, and if it's a special day. The goal is to make parking lots used better — not too crowded or empty.
 Data I Used

The dataset has details of 14 parking lots. It includes:

    Time and date

    How many vehicles are parked

    How many are waiting (queue)

    Type of vehicle – car, bike, truck

    Traffic nearby

    Whether it's a special day or not

 Models I Made
 Model 1: Simple (Baseline) Model

This model increases the price a bit when the lot starts getting full.
It starts from ₹10 and keeps increasing slowly with occupancy.
Model 2: Smarter (Demand-Based) Model

This one looks at many things:

    How full the lot is

    Queue length

    Traffic

    Special day

    Type of vehicle

I made a formula to calculate "demand" and changed the price based on that.
If demand is high → price increases
If demand is low → price decreases

I also made sure price stays within a proper range (not too high or low).
Assumptions I Made

    More occupancy and queue means more demand

    If it's a holiday or special event, demand goes up

    If traffic is too much, people avoid parking, so demand drops

    Trucks need more space, so they're given higher weight

    I didn’t use competitor prices (skipped Model 3 for now)

How Prices Change

    Lot full + long queue + special day = higher price

    Empty lot or too much traffic = lower price

    Demand model changes prices smoothly and works better

Visualizations I Made

I used Bokeh to show two lines:

    Green line = Simple model price

    Blue line = Demand-based model price

This helps to compare and see how prices change with time.

Final Thoughts

I learned how small things like traffic, vehicle type, and holidays can affect pricing. I used only basic Python libraries (pandas, numpy, bokeh) and wrote everything myself. This model can be made even better by adding competition logic or live rerouting later.